# Time series analysis of sentiments in period of time

In [47]:
import pandas as pd
import pmdarima
import numpy as np
import statsmodels.tsa.arima_model as stm
import plotly.express as px
import plotly.graph_objs as go
import kaleido
import matplotlib.pyplot as plt

## Loading data

In [39]:
df_1 = pd.read_csv('tweets_sentiments.csv')
df_2 = pd.read_csv('output.csv')

In [40]:
df_1.head(10)

,Text,Tension
0,bardzo ciekawa według miąć opracować,0
1,jeden z powód dla który wysyłać tak duży pomoc...,-1
2,przekazać jugosłowiański czołg bojowy ten wari...,0
3,przekazać jugosłowiański czołg bojowy ten wari...,0
4,clip,0
5,minuta w parlament europejski,0
6,przekazać jugosłowiański czołg bojowy ten wari...,0
7,przekazać jugosłowiański czołg bojowy ten wari...,0
8,zgodzić siebie na zakup ciężki bronić do bezpł...,0
9,minuta w parlament europejski,0


In [41]:
df_2.head(10)

,Unnamed: 0,id,text,retweet_count,created_at,author,location,is_verified
0,0.0,1517289557741477890,"Bardzo ciekawe, według mnie, opracowanie #Ukra...",0,2022-04-21 23:49:55+00:00,AntoniPiechocki,NaN,False
1,1.0,1517284025928495104,"RT @propeertys: Jednym z powodów, dla których ...",18,2022-04-21 23:27:56+00:00,52_macius,Poznań Stare Miasto / EU,False
2,2.0,1517281903241482241,RT @propeertys: 🇸🇮Słowenia przekaże 🇺🇦Ukrainie...,17,2022-04-21 23:19:30+00:00,KozlowskiMarek,Wrocław,False
3,3.0,1517281417943736321,RT @propeertys: 🇸🇮Słowenia przekaże 🇺🇦Ukrainie...,17,2022-04-21 23:17:34+00:00,lucinda_10,NaN,False
4,4.0,1517280783345635329,.2345z #PeerTube CLIP https://t.co/OnDvp8AxN0 ...,0,2022-04-21 23:15:03+00:00,BastardHegels,NaN,False
5,5.0,1517280086541717504,RT @dabrowska_hf: 7 minut w Parlamencie Europe...,9,2022-04-21 23:12:16+00:00,karolk18,NaN,False
6,6.0,1517279847885774852,RT @propeertys: 🇸🇮Słowenia przekaże 🇺🇦Ukrainie...,17,2022-04-21 23:11:20+00:00,MarioRebeliant,Poland,False
7,7.0,1517279637080100865,RT @propeertys: 🇸🇮Słowenia przekaże 🇺🇦Ukrainie...,17,2022-04-21 23:10:29+00:00,52_macius,Poznań Stare Miasto / EU,False
8,8.0,1517279617647882241,RT @propeertys: UE zgodziła się na zakup ciężk...,44,2022-04-21 23:10:25+00:00,djankowski10,NaN,False
9,9.0,1517279343763984384,RT @dabrowska_hf: 7 minut w Parlamencie Europe...,9,2022-04-21 23:09:19+00:00,RenataJasinskaG,"Płock, Polska",False


In [79]:
# We should take created_at column from df_2 and Tension columns from df_1 and create new data frame which will be time series
ts = pd.DataFrame(data = np.array(df_1['Tension']), index = df_2['created_at'], columns = ['Tension'])
ts = ts.sort_index(ascending = True)
ts.head(10)

,Tension
created_at,
2022-04-15 21:02:01+00:00,-1
2022-04-15 21:02:23+00:00,-1
2022-04-15 21:04:04+00:00,0
2022-04-15 21:04:19+00:00,0
2022-04-15 21:04:21+00:00,-1
2022-04-15 21:05:14+00:00,-1
2022-04-15 21:05:17+00:00,-1
2022-04-15 21:05:58+00:00,0
2022-04-15 21:06:49+00:00,1


In [80]:
# We need to make moving sum

def moving_sum(pd_series):
    """Function returns array-like object with moving sum of pd_series"""
    
    s = 0
    result = []
    
    for element in pd_series:
        s += element
        result.append(s)
        
    return result

ts['moving_tension'] = moving_sum(ts['Tension'])

del ts['Tension']

In [81]:
ts.head(10)

,moving_tension
created_at,
2022-04-15 21:02:01+00:00,-1
2022-04-15 21:02:23+00:00,-2
2022-04-15 21:04:04+00:00,-2
2022-04-15 21:04:19+00:00,-2
2022-04-15 21:04:21+00:00,-3
2022-04-15 21:05:14+00:00,-4
2022-04-15 21:05:17+00:00,-5
2022-04-15 21:05:58+00:00,-5
2022-04-15 21:06:49+00:00,-4


In [82]:
fig = go.Figure(data=go.Scatter(x=ts.index, 
                        y=ts['moving_tension'],
                        marker_color='indianred', text="tension"))
fig.update_layout({"title": 'Tweets',
                   "xaxis": {"title":"Days"},
                   "yaxis": {"title":"Total tension"},
                   "showlegend": False})
fig.show()